In [1]:
#FWI MAP
###Import tools
from input_fwi import GFS_MAP
from fwi_calculator import FWICalculator

In [2]:
###Authenticates and initializes Earth Engine
import geemap as geemap
import ee
import eemont
import ipyleaflet
import datetime

ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AfgeXvvOlD38c8Vn3scroJ4bFytXoHtmxf3IcyUtWbQtnQVhXFiXWHzjgJA

Successfully saved authorization token.


In [3]:
###Input date, timezone and boundary of data
date = datetime.date.today() - datetime.timedelta(days=1)
timezone = 'Asia/Jakarta'
boundary = ee.Geometry.BBox(90, -15.0, 150.0, 15.0)

###Calculate FWI
inputs = GFS_MAP(date, timezone, boundary)
calculator = FWICalculator(date, inputs)
calculator.set_previous_codes()
fwi = calculator.compute()

###Create color palette
vis_params = {
    'min' : 0,
    'max' : 64,
    'palette' : ['blue', 'green', 'yellow', 'red']
}

###Select boundary of map that will show in Map
###For Indonesia boundary, we select from FAO dataset
indonesia = ee.FeatureCollection('FAO/GAUL/2015/level0') \
    .filter(ee.Filter.eq('ADM0_NAME', 'Indonesia'))
fwi_map = fwi.clip(indonesia)

In [22]:
###Visualize the map
Map = geemap.Map(center=[0, 120], zoom= 4)
Map.addLayer(fwi_map, vis_params, 'FWI');
Map

Map(center=[0, 120], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [5]:
###Create geometry boundary to export the image
indo_shape = indonesia.geometry()

In [6]:
###Import os to input output path and fime name
import os
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'FWI.tif')

In [7]:
###Export map
geemap.ee_export_image(fwi, filename=filename, scale=1500, region=indo_shape, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Nuraini\Downloads\FWI.tif


In [27]:
#SPI MAP
### Set time of monthly length
start = 31+28

### Set time frame of running and download
start_date = '1981-01-01'
end_date = '2021-12-31'

start_Download = '2022-01-01'
end_Download = '2022-11-01'

### Set resolution of our map
resolution = 500

### Long term average (LTA) can be generated directly from the pre-calculated Rainfall
lta = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filter(ee.Filter.dayOfYear(start, start + 31)).filterDate(start_date, end_date)

lta_mean = lta.select('precipitation').mean().clip(indonesia)
lta_std = lta.reduce(ee.Reducer.stdDev()).clip(indonesia)

### the latest rainfall
latest = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filter(ee.Filter.dayOfYear(start, start + 31)).filterDate(start_Download, end_Download)

rainfall_now = latest.select('precipitation').mean().clip(indonesia)


### Calculating rainfall anomaly and standardized precipitation index 
anomaly = rainfall_now.divide(lta_mean).multiply(100)
spi = rainfall_now.subtract(lta_mean).divide(lta_std)

#Define a colour table
red2blue = ['a60027','de3e2c','fa8b50','ffd485','e1e1e1','d3edf5','90c4de','5183bd','313694']
rainfall_viz = {min:0, max:25, 'palette': ['B8810B', 'CB8C08', 'FFE47E','FFF2C1', 'DAF7A6', 'D2FFFF', '2DE7E7', '00B9D6', '008BD6', '006399']}
rainanom_viz = {min:0, max:200, 'palette': red2blue}
spicolor = ['0000aa','0000ff','00aaff','00ffff','aaff55','ffffff','ffff00','fcd37f','ffaa00','e60000','730000']
spi_viz = {min:-2, max:2, 'palette': spicolor}



In [28]:
### Create Map
print('The observed time period of monthly SPI starts on ',
      start_date.format('YYYY-MM-dd'),
      'and ends on',
      end_date.format('YYYY-MM-dd'))

Map1 = geemap.Map(center=[0, 120], zoom= 4)

### Add the image to the map using both the color ramp and interval schemes.
Map1.addLayer(anomaly, rainanom_viz, 'Anomaly (Indo2022-LTA)')
Map1.addLayer(spi, spi_viz, 'SPI Indo2022')
Map1

The observed time period of monthly SPI starts on  1981-01-01 and ends on 2021-12-31


Map(center=[0, 120], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…